In [2]:
%pip install --disable-pip-version-check torch==1.13.1 torchdata==0.5.1

Note: you may need to restart the kernel to use updated packages.


In [3]:
!nvidia-smi

Mon Apr 24 00:50:01 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A10G         Off  | 00000000:00:1E.0 Off |                    0 |
|  0%   29C    P0    60W / 300W |   7255MiB / 22731MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## _==> Please ignore all WARNINGs and ERRORs from the `pip install`'s above. <==_

# Hugging Face Transformers
## _==> Please ignore all WARNINGs and ERRORs from the `pip install`'s below. <==_

In [4]:
%pip install --disable-pip-version-check -Uq \
    transformers==4.27.2 \
    datasets==2.9.0 \
    accelerate \
    promptsource==0.2.3 \
    evaluate==0.4.0 \
    rouge_score==0.1.2 \
    loralib==0.1.1 

Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install git+https://github.com/huggingface/peft.git

  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-pqjv0ahv
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-pqjv0ahv
  Resolved https://github.com/huggingface/peft.git to commit 2822398fbe896f25d4dac5e468624dc5fd65a51b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for peft: filename=peft-0.3.0.dev0-py3-none-any.whl size=50481 sha256=9a4e8b1a722e91b1319530695e2540f8727da86631d62bc4e561ff957a0f992a
  Stored in directory: /tmp/pip-ephem-wheel-cache-x2u5v5ky/wheels/13/73/6d/ff27a3703d8bad21d7e0c24cbd9dde5d7ae78f756405707a0c
Successfully built peft

[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install git+https://github.com/lvwerra/trl.git

  Cloning https://github.com/lvwerra/trl.git to /tmp/pip-req-build-pywl2pwb
  Running command git clone --filter=blob:none --quiet https://github.com/lvwerra/trl.git /tmp/pip-req-build-pywl2pwb
  Resolved https://github.com/lvwerra/trl.git to commit ce37eadcfa22f2a3c25422411a586b8f593e3e6e
  Preparing metadata (setup.py) ... done

[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
import argparse
import os

import torch
from accelerate import Accelerator
from datasets import load_dataset
from peft import LoraConfig, get_peft_model #, prepare_model_for_int8_training
from torch.utils.data import IterableDataset
from tqdm import tqdm
from transformers import AutoConfig, AutoModelForSeq2SeqLM, AutoTokenizer, Trainer, TrainingArguments, logging, set_seed

In [8]:
def get_prompt_with_response(example):
    text = f"Question: {example['question']}\n\nAnswer: {example['response_j']}"
    return text

def get_prompt_from_question(question):
    text = f"Question: {question}\n\nAnswer: "
    return text

def get_prompt_without_response(example):
    text = f"Question: {example['question']}\n\nAnswer: "
    return text

def get_response_without_prompt(example):
    return example['response_j']


def chars_token_ratio(dataset, tokenizer, nb_examples=400):
    """
    Estimate the average number of characters per token in the dataset.
    """
    total_characters, total_tokens = 0, 0
    for _, example in tqdm(zip(range(nb_examples), iter(dataset)), total=nb_examples):
        text = get_prompt_with_response(example)
        total_characters += len(text)
        if tokenizer.is_fast:
            total_tokens += len(tokenizer(text).tokens())
        else:
            total_tokens += len(tokenizer.tokenize(text))

    return total_characters / total_tokens


def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

    
class ConstantLengthDataset(IterableDataset):
    """
    Iterable dataset that returns constant length chunks of tokens from stream of text files.
        Args:
            tokenizer (Tokenizer): The processor used for proccessing the data.
            dataset (dataset.Dataset): Dataset with text files.
            infinite (bool): If True the iterator is reset after dataset reaches end else stops.
            seq_length (int): Length of token sequences to return.
            num_of_sequences (int): Number of token sequences to keep in buffer.
            chars_per_token (int): Number of characters per token used to estimate number of tokens in text buffer.
    """

    def __init__(
        self,
        tokenizer,
        dataset,
        infinite=False,
        seq_length=1024,
        num_of_sequences=1024,
        chars_per_token=3.6,
    ):
        self.tokenizer = tokenizer
        self.concat_token_id = tokenizer.eos_token_id if tokenizer.eos_token_id else eos_token_id
        self.dataset = dataset
        self.seq_length = seq_length
        self.infinite = infinite
        self.current_size = 0
        self.max_buffer_size = seq_length * chars_per_token * num_of_sequences

    def __iter__(self):
        iterator = iter(self.dataset)
        more_examples = True
        while more_examples:
            buffer, buffer_len = [], 0
            while True:
                if buffer_len >= self.max_buffer_size:
                    break
                try:
                    buffer.append(get_prompt_with_response(next(iterator)))
                    buffer_len += len(buffer[-1])
                except StopIteration:
                    if self.infinite:
                        iterator = iter(self.dataset)
                    else:
                        more_examples = False
                        break
            tokenized_inputs = self.tokenizer(buffer, truncation=False)["input_ids"]
            all_token_ids = []
            for tokenized_input in tokenized_inputs:
                all_token_ids.extend(tokenized_input + [self.concat_token_id])
            for i in range(0, len(all_token_ids), self.seq_length):
                input_ids = all_token_ids[i : i + self.seq_length]
                if len(input_ids) == self.seq_length:
                    self.current_size += 1
                    yield {
                        "input_ids": torch.LongTensor(input_ids),
                        "labels": torch.LongTensor(input_ids),
                    }

# Fine-Tune flan-t5 on stack exchange paired dataset

In [14]:
model_path="google/flan-t5-base"
dataset_name="lvwerra/stack-exchange-paired"
subset="data/finetune"
split="train"
size_valid_set=4000
streaming=False
shuffle_buffer=5000
seq_length=1024
max_steps=500 # was 5000
batch_size=2 # was 16
gradient_accumulation_steps=1
eos_token_id=49152
learning_rate=1e-5
lr_scheduler_type="cosine"
num_warmup_steps=50 # was 100
weight_decay=0.05
local_rank=0
no_fp16=True
bf16=True
no_gradient_checkpointing=True
seed=42
num_workers=None
log_freq=100
eval_freq=100 # was 1000
save_freq=100 # was 1000

In [15]:
set_seed(seed)

tokenizer = AutoTokenizer.from_pretrained(model_path)

print('Dataset name: {}'.format(dataset_name))
dataset = load_dataset(
    dataset_name,
    data_dir=subset,
    split=split,
    num_proc=num_workers if not streaming else None,
    streaming=streaming,
)

print(dataset)
if streaming:
    print("Loading the dataset in streaming mode")
    valid_data = dataset.take(size_valid_set)
    train_data = dataset.skip(size_valid_set)
    train_data = train_data.shuffle(buffer_size=shuffle_buffer, seed=seed)
else:
    dataset = dataset.train_test_split(test_size=0.005, seed=seed)
    train_data = dataset["train"]
    valid_data = dataset["test"]
    print(f"Size of the train set: {len(train_data)}. Size of the validation set: {len(valid_data)}")

chars_per_token = chars_token_ratio(train_data, tokenizer)
print(f"The character to token ratio of the dataset is: {chars_per_token:.2f}")

train_dataset = ConstantLengthDataset(
    tokenizer,
    train_data,
    infinite=True,
    seq_length=seq_length,
    chars_per_token=chars_per_token,
)
validation_dataset = ConstantLengthDataset(
    tokenizer,
    valid_data,
    infinite=False,
    seq_length=seq_length,
    chars_per_token=chars_per_token,
)

print(train_dataset)
print(validation_dataset)

Dataset name: lvwerra/stack-exchange-paired


Using custom data configuration lvwerra--stack-exchange-paired-4e8354da9bd1bd3d
Found cached dataset parquet (/root/.cache/huggingface/datasets/lvwerra___parquet/lvwerra--stack-exchange-paired-4e8354da9bd1bd3d/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Loading cached split indices for dataset at /root/.cache/huggingface/datasets/lvwerra___parquet/lvwerra--stack-exchange-paired-4e8354da9bd1bd3d/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-54456b7e473dd794.arrow and /root/.cache/huggingface/datasets/lvwerra___parquet/lvwerra--stack-exchange-paired-4e8354da9bd1bd3d/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-8304e884db3f8548.arrow


Dataset({
    features: ['qid', 'question', 'date', 'metadata', 'response_j', 'response_k'],
    num_rows: 7440923
})
Size of the train set: 7403718. Size of the validation set: 37205


100%|██████████| 400/400 [00:00<00:00, 588.44it/s]

The character to token ratio of the dataset is: 3.08


# Fine-tune Model with PEFT

In [16]:
# disable caching mechanism when using gradient checkpointing
base_model = AutoModelForSeq2SeqLM.from_pretrained('google/flan-t5-base',
                                                   use_cache=not no_gradient_checkpointing,
                                                   #torch_dtype=torch.float16
                                                  )
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_2_SEQ_LM",
)

peft_model = get_peft_model(base_model, lora_config)
print_trainable_parameters(peft_model)

trainable params: 1769472 || all params: 249347328 || trainable%: 0.7096414524241463


In [17]:
peft_fine_tuned_with_public_qanda_checkpoint='./peft_fine_tuned_with_public_qanda'

In [19]:
peft_training_args = TrainingArguments(
    output_dir=peft_fine_tuned_with_public_qanda_checkpoint,
    dataloader_drop_last=True,
#    evaluation_strategy="steps",    
    max_steps=max_steps,
#    eval_steps=eval_freq,
#    save_steps=save_freq,
    logging_steps=log_freq,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=learning_rate,
    lr_scheduler_type=lr_scheduler_type,
    warmup_steps=num_warmup_steps,
    gradient_accumulation_steps=gradient_accumulation_steps,
    gradient_checkpointing=not no_gradient_checkpointing,
    fp16=not no_fp16,
    bf16=bf16,
    weight_decay=weight_decay
)

peft_trainer = Trainer(model=peft_model, 
                  args=peft_training_args, 
                  train_dataset=train_dataset, 
                  #eval_dataset=validation_dataset
                )

print("Training...")
peft_trainer.train()

Training...


Step,Training Loss
1,1.254600
2,1.251500
3,1.018500
4,2.172700
5,1.731400
6,1.486300
7,1.839000
8,2.191000
9,1.624300
10,1.577400


TrainOutput(global_step=500, training_loss=0.9616686029434204, metrics={'train_runtime': 214.0039, 'train_samples_per_second': 4.673, 'train_steps_per_second': 2.336, 'total_flos': 1380386340864000.0, 'train_loss': 0.9616686029434204, 'epoch': 1.0})

In [29]:
print("Saving last checkpoint of the peft model")
#merged_model=peft_trainer.model.merge_and_unload()
peft_trainer.model.save_pretrained(peft_fine_tuned_with_public_qanda_checkpoint)
tokenizer.save_pretrained(peft_fine_tuned_with_public_qanda_checkpoint)

Saving last checkpoint of the peft model


('./peft_fine_tuned_with_public_qanda/tokenizer_config.json',
 './peft_fine_tuned_with_public_qanda/special_tokens_map.json',
 './peft_fine_tuned_with_public_qanda/tokenizer.json')

In [30]:
%store peft_fine_tuned_with_public_qanda_checkpoint

Stored 'peft_fine_tuned_with_public_qanda_checkpoint' (str)


In [31]:
from peft import PeftModel

peft_model = PeftModel.from_pretrained(base_model, peft_fine_tuned_with_public_qanda_checkpoint)
peft_model_tokenizer = AutoTokenizer.from_pretrained(peft_fine_tuned_with_public_qanda_checkpoint)

# Qualitative results

In [36]:
question = 'Who won the 2022 World Cup?'

prompt = get_prompt_from_question(question)

inputs = peft_model_tokenizer(prompt, return_tensors='pt')
response = peft_model_tokenizer.decode(
    peft_model.generate(
       input_ids= inputs["input_ids"], 
       max_new_tokens=200,
    )[0], 
    skip_special_tokens=True
)
print(f'INPUT PROMPT:\n{prompt}{response}\n')

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper__index_select)

In [37]:
print('Dataset name: {}'.format(dataset_name))
dataset = load_dataset(
    dataset_name,
    data_dir='data/evaluation',
    split='train',
)

print(dataset)

Dataset name: lvwerra/stack-exchange-paired


Using custom data configuration lvwerra--stack-exchange-paired-b55353e62cc7a74b
Found cached dataset parquet (/root/.cache/huggingface/datasets/lvwerra___parquet/lvwerra--stack-exchange-paired-b55353e62cc7a74b/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


Dataset({
    features: ['qid', 'question', 'date', 'metadata', 'response_j', 'response_k'],
    num_rows: 4483004
})


In [38]:
prompts_and_responses = dataset \
    .select(range(10)).map(lambda example : {
        'prompt': get_prompt_without_response(example), 
        'response': get_response_without_prompt(example)
    })
prompts_and_responses[0]['question']
prompts_and_responses[0]['response_j']

Loading cached processed dataset at /root/.cache/huggingface/datasets/lvwerra___parquet/lvwerra--stack-exchange-paired-b55353e62cc7a74b/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-ff5d9e4fd03dc106.arrow


"> \n> Making my Java program easily distributable\n> \n> \n> \n\nIf you mean 'easy for the end user' look to [Java Web Start](https://stackoverflow.com/tags/java-web-start/info).\n\n---\n\nA passer-by asks:\n\n> \n> Can you package the dll dependencies with Web Start? \n> \n> \n> \n\nYes, but much, much better. You can package the natives for each platform in separate Jars, and supply them only to the platform that uses that native, even so far as partitioning the download between 32 & 64 bit versions of the natives.\n\nJWS puts the natives on the run-time class-path of the application, ready for loading in code.\n\nThis all happens automatically for the end user, they click a link, approve the trust dialog(s) when asked, and the application installs - possibly with desktop integration, and appears on screen like magic.\n\nJWS apps. that use natives need to be distributed as `all-permissions` security level, because the JVM cannot guarantee the actions of anything that 'goes native'."

In [39]:
for prompt_response in prompts_and_responses:
    prompt = prompt_response['prompt']
    inputs = peft_model_tokenizer(prompt, return_tensors='pt')

    response = peft_model_tokenizer.decode(
                peft_model.generate(
                    input_ids=inputs["input_ids"], 
                    max_new_tokens=200,
                    do_sample=True, 
                    top_k=50, 
                    top_p=0.9
                    )[0],
                    skip_special_tokens=True)

    print('PROMPT: {}'.format(prompt))
    print('RESPONSE: {}'.format(response))
    print('EXPECTED RESPONSE: {}'.format(prompt_response['response']))
    print('----')

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper__index_select)

# Quantitative Results with ROGUE Metric

The [ROUGE metric](https://en.wikipedia.org/wiki/ROUGE_(metric)) helps quantify the validity of summarizations produced by models. It compares summarizations to a "baseline" summary which is usually created by a human. While not perfect, it does give an indication to the overall increase in summarization effectiveness that we have accomplished by fine-tuning.# ROUGE evaluation of summaries

In [40]:
import evaluate

rouge = evaluate.load('rouge')

In [41]:
prompts = prompts_and_responses['prompt']
human_baseline_summaries = prompts_and_responses['response']
human_baseline_summaries

["> \n> Making my Java program easily distributable\n> \n> \n> \n\nIf you mean 'easy for the end user' look to [Java Web Start](https://stackoverflow.com/tags/java-web-start/info).\n\n---\n\nA passer-by asks:\n\n> \n> Can you package the dll dependencies with Web Start? \n> \n> \n> \n\nYes, but much, much better. You can package the natives for each platform in separate Jars, and supply them only to the platform that uses that native, even so far as partitioning the download between 32 & 64 bit versions of the natives.\n\nJWS puts the natives on the run-time class-path of the application, ready for loading in code.\n\nThis all happens automatically for the end user, they click a link, approve the trust dialog(s) when asked, and the application installs - possibly with desktop integration, and appears on screen like magic.\n\nJWS apps. that use natives need to be distributed as `all-permissions` security level, because the JVM cannot guarantee the actions of anything that 'goes native'.

In [42]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

pretrained_model_checkpoint='t5-base'
pretrained_model_tokenizer = AutoTokenizer.from_pretrained(pretrained_model_checkpoint, use_fast=True)
pretrained_model = AutoModelForSeq2SeqLM.from_pretrained(pretrained_model_checkpoint)

instruct_model_checkpoint='google/flan-t5-base'
instruct_model_tokenizer = AutoTokenizer.from_pretrained(instruct_model_checkpoint, use_fast=True)
instruct_model = AutoModelForSeq2SeqLM.from_pretrained(instruct_model_checkpoint)

In [43]:
from transformers import GenerationConfig

pretrained_model_summaries = []
instruct_model_summaries = []
#supervised_fine_tuned_model_summaries = []
peft_model_summaries = []

for idx, prompt in enumerate(prompts):
    input_ids = pretrained_model_tokenizer(prompt, return_tensors="pt").input_ids

    pretrained_model_outputs = pretrained_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    pretrained_model_text_output = pretrained_model_tokenizer.decode(pretrained_model_outputs[0], skip_special_tokens=True)
    pretrained_model_summaries.append(pretrained_model_text_output)

    instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    instruct_model_text_output = instruct_model_tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)
    instruct_model_summaries.append(instruct_model_text_output)
    
    # supervised_fine_tuned_model_outputs = supervised_fine_tuned_model_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    # supervised_fine_tuned_model_text_output = supervised_fine_tuned_model_model_tokenizer.decode(supervised_fine_tuned_model_outputs[0], skip_special_tokens=True)
    # supervised_fine_tuned_model_summaries.append(supervised_fine_tuned_model_model_text_output)
    
    peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    peft_model_text_output = peft_model_tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)
    peft_model_summaries.append(peft_model_text_output)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper__index_select)

In [ ]:
pretrained_model_results = rouge.compute(
    predictions=pretrained_model_summaries,
    references=human_baseline_summaries[0:len(pretrained_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)
pretrained_model_results

In [ ]:
instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)
instruct_model_results

In [ ]:
# supervised_fine_tuned_model_results = rouge.compute(
#     predictions=supervised_fine_tuned_model_summaries,
#     references=human_baseline_summaries[0:len(supervised_fine_tuned_model_summaries)],
#     use_aggregator=True,
#     use_stemmer=True,
# )
# supervised_fine_tuned_model_results

In [ ]:
peft_model_results = rouge.compute(
    predictions=peft_loaded_model_summaries,
    references=human_baseline_summaries[0:len(peft_loaded_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)
peft_model_results